In [ ]:
# For Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

In [ ]:
master_numeric = pd.read_csv('master_numeric_mean.csv')

In [ ]:
master_numeric["('mean', 'so2')"] = master_numeric["('mean', 'so2')"]*100

In [ ]:
master_numeric.sort_values(by=['date'],inplace=True)

In [ ]:
master_numeric.drop(["('mean', 'humidity')","('mean', 'temperature')"],axis=1,inplace=True)

In [ ]:
B_lockdown1 = master_numeric['date'].between('2020-02-15','2020-03-17')
Lockdown1 = master_numeric['date'].between('2020-03-17','2020-05-10')

B_lockdown2 = master_numeric['date'].between('2020-05-10','2020-10-17')
Lockdown2 = master_numeric['date'].between('2020-10-17','2020-12-14')

B_lockdown3 = master_numeric['date'].between('2020-12-14','2021-02-26')
Lockdown3 = master_numeric['date'].between('2021-02-26','2021-05-02')

A_lockdown3 = master_numeric['date'].between('2021-05-02','2021-07-27')

master_numeric['lockdown'] = np.select([B_lockdown1, Lockdown1, B_lockdown2, Lockdown2, B_lockdown3, Lockdown3, A_lockdown3], 
                                       ['before_lockdown1','lockdown_1','before_lockdown2','lockdown_2','before_lockdown3','lockdown_3','after_lockdown3'], 
                                       default='No lockdown')

In [ ]:
master_numeric.columns

In [ ]:
master_numeric.columns  = ['date', 'region', 'lockdown',
       'retail_and_recreation',
       'grocery_and_pharmacy',
       'residential', 'driving', 'walking',
       'transit', 'geometry','no2', 'o3', 'pm10','pm25', 'pressure', 'so2'
                          ]

In [ ]:
master_numeric.region.unique()

### functions

In [ ]:
def stacked_chart(dataset,color_no2,color_o3,color_pm10,color_pm25,color_so2):

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dataset.date, y=dataset.driving,
        hoverinfo='x+y',
        mode='lines',
        name='driving',opacity=0.5,
        line=dict(width=0.5, color='rgb(0, 100, 0)'),
        stackgroup='one'))
    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset.grocery_and_pharmacy)+100,
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(255,0,255)'),
        stackgroup='one',opacity=0.5,
        name='grocery and pharmacy'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset.retail_and_recreation)+100,
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(47,79,79)'),
        stackgroup='one',opacity=0.5,
        name='retail and recreation'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset.residential)+100,
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, color='rgb(75,0,130)'),
        stackgroup='one',opacity=0.5,
        name='residential'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset["so2"]*100),mode='markers',
        marker=dict(color=color_so2),
        hoverinfo='x+y',
    #     line=dict(width=0.5, color='rgb(256, 50, 100)'),
    #     stackgroup='one',
        name='so2'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset["no2"]*100),mode='markers',
        marker=dict(color=color_no2),
        hoverinfo='x+y',
    #     line=dict(width=0.5, color='rgb(256, 50, 100)'),
    #     stackgroup='one',
        name='no2'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset["o3"]*100),mode='markers',
        marker=dict(color=color_o3),
        hoverinfo='x+y',
    #     line=dict(width=0.5, color='rgb(256, 50, 100)'),
    #     stackgroup='one',
        name='o3'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset["pm10"]*100),mode='markers',
        marker=dict(color=color_pm10),
        hoverinfo='x+y',
    #     line=dict(width=0.5, color='rgb(256, 50, 100)'),
    #     stackgroup='one',
        name='pm10'))

    fig.add_trace(go.Scatter(
        x=dataset.date, y=(dataset["pm25"]*100),mode='markers',
        marker=dict(color=color_pm25),
        hoverinfo='x+y',
    #     line=dict(width=0.5, color='rgb(256, 50, 100)'),
    #     stackgroup='one',
        name='pm25'))

    
    fig.add_vrect(x0=dataset[dataset.lockdown== 'lockdown_1']['date'].iloc[0],
                  x1=dataset[dataset.lockdown== 'lockdown_1']['date'].iloc[-1],annotation_text="lockdown 1")
    fig.add_vrect(x0=dataset[dataset.lockdown== 'lockdown_2']['date'].iloc[0],
                  x1=dataset[dataset.lockdown== 'lockdown_2']['date'].iloc[-1],annotation_text="lockdown 2")
    fig.add_vrect(x0=dataset[dataset.lockdown== 'lockdown_3']['date'].iloc[0],
                  x1=dataset[dataset.lockdown== 'lockdown_3']['date'].iloc[-1],annotation_text="lockdown 3")


    # fig = px.area(data_frame=marseille,x='date',y='driving')
    # fig.add_trace(px.area(data_frame=marseille,x='date',y="(\'mean\', \'so2\')"))
    # fig.update_layout(yaxis_range=(0, 1000))
    fig.update_xaxes(rangeslider_visible=True)
    fig.show()

In [ ]:
def linear_to_quality_catagory(dataset,col,very_low,low,medium,high,very_high):
    
    air_pollutant = []
    
    for j in range(len(dataset[col])):

        if dataset[col].values[j] < very_low:
            dataset[col].iloc[j] = 1
            air_pollutant.append('green')
        
        elif very_low < dataset[col].values[j] < low:
            dataset[col].iloc[j] = 2
            air_pollutant.append('yellow')
        
        elif low < dataset[col].values[j] < medium:
            dataset[col].iloc[j] = 3
            air_pollutant.append('orange')
        
        elif medium < dataset[col].values[j] < high:
            dataset[col].iloc[j] = 4
            air_pollutant.append('red')
        
        elif high < dataset[col].values[j] < very_high:
            dataset[col].iloc[j] = 5
            air_pollutant.append('brown')
        
        elif very_high<=dataset[col].values[j]:
            dataset[col].iloc[j] = 6
            air_pollutant.append('black')
        else:
            dataset[col].iloc[j] = 0
            air_pollutant.append('gray')

    return dataset[col],air_pollutant

In [ ]:
def air_pollutant_catagory_color(dataset):
    dataset["no2"],color_no2 = linear_to_quality_catagory(dataset,"no2",10,20,35,50,100)
    dataset["so2')"],color_so2 = linear_to_quality_catagory(dataset,"so2",25,50,120,350,500)
    dataset["pm10"],color_pm10 = linear_to_quality_catagory(dataset,"pm10",10,20,35,50,100)
    dataset["pm25"],color_pm25 = linear_to_quality_catagory(dataset,"pm25",5,10,20,25,60)
    dataset["o3"],color_o3 = linear_to_quality_catagory(dataset,"o3",33,65,120,180,240)
    
    return dataset,color_no2,color_o3,color_pm10,color_pm25,color_so2

In [ ]:
def gauge_mobility_lockdowns(dataset,column1,lockdown,x1,x2,y1,y2):
    fig_g.add_trace(go.Indicator(
        mode = "gauge+number",
        delta = {'reference': dataset[column1].mean()},
        value = dataset[dataset.lockdown== lockdown][column1].mean(),
        title = {'text': lockdown,'font' : {'size' : 13}},
        domain = {'x': [x1, x2], 'y': [y1, y2]},
        gauge = {
        'axis': {'range': [None, dataset[column1].quantile(1.0)], 'tickwidth': 1, 'tickcolor': "gray"},
        'bar': {'color': "black"},
        'bgcolor': "white",
        'borderwidth': 0.01,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 
                       dataset[column1].quantile(0.25)], 'color': 'orange'},
            {'range': [dataset[column1].quantile(0.25), 
                       dataset[column1].quantile(0.50)], 'color': 'yellow'},
            {'range': [dataset[column1].quantile(0.50),
                       dataset[column1].quantile(0.75)], 'color': 'red'},
            {'range': [dataset[column1].quantile(0.75),
                      dataset[column1].quantile(1.0)], 'color': 'gray'}]
        }
            ))
    return fig_g



In [ ]:
def gauge_air_pollutant_lockdowns(dataset,column2,lockdown,x1,x2,y1,y2):
    fig_g.add_trace(go.Indicator(
        mode = "gauge+number",
        delta = {'reference': dataset[column2].mean()},
        value = dataset[dataset.lockdown== lockdown][column2].mean(),
        title = {'text': lockdown,'font' : {'size' : 13}},
        domain = {'x': [x1, x2], 'y': [y1, y2]},
        gauge = {
        'axis': {'range': [None, dataset[column2].quantile(1.0)], 'tickwidth': 1, 'tickcolor': "gray"},
        'bar': {'color': "black"},
        'bgcolor': "white",
        'borderwidth': 0.1,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 
                       1], 'color': 'ghostwhite'},
            {'range': [1, 
                       2], 'color': 'lavender'},
            {'range': [2,
                       3], 'color': 'cornflowerblue'},
            {'range': [3,
                      4], 'color': 'royalblue'},
            {'range': [4,
                       5], 'color': 'blue'},
            {'range': [5,
                       6], 'color': 'navy'}]
        }
            ))
    return fig_g



In [ ]:
def gauge_compare(dataset,column1,column2):
    
#################### before lockdown1 ############################
    fig_b = gauge_mobility_lockdowns(dataset,column1,'before_lockdown1',0,0.2,0.9,1)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'before_lockdown1',0.2,0.4,0.9,1)

#################### lockdown 1 ############################

    fig_b = gauge_mobility_lockdowns(dataset,column1,'lockdown_1',0,0.2,0.65,0.75)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'lockdown_1',0.2,0.4,0.65,0.75)

#################### before lockdown2 ############################

    fig_b = gauge_mobility_lockdowns(dataset,column1,'before_lockdown2',0.6,0.8,0.7,0.8)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'before_lockdown2',0.8,1,0.7,0.8)

#################### lockdown 2 ############################
    fig_b = gauge_mobility_lockdowns(dataset,column1,'lockdown_2',0.6,0.8,0.4,0.5)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'lockdown_2',0.8,1,0.4,0.5)

# ####################before lockdown3 ############################

    fig_b = gauge_mobility_lockdowns(dataset,column1,'before_lockdown3',0.1,0.3,0.3,0.4)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'before_lockdown3',0.3,0.5,0.3,0.4)

# #################### lockdown 3 ############################

    fig_b = gauge_mobility_lockdowns(dataset,column1,'lockdown_3',0.1,0.3,0,0.1)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'lockdown_3',0.3,0.5,0,0.1)

# #################### after lockdown 3 ############################

    fig_b = gauge_mobility_lockdowns(dataset,column1,'after_lockdown3',0.5,0.7,0,0.1)
    fig_b = gauge_air_pollutant_lockdowns(dataset,column2,'after_lockdown3',0.7,0.9,0,0.1)

    fig_b.update_layout(title=column1+' vs '+column2)
    fig_b.show()

In [ ]:
def bulleten_mobility_lockdowns(dataset,column1,lockdown,x1,x2,y1,y2):
    fig_b.add_trace(go.Indicator(
        mode = "number+gauge+delta",
        delta = {'reference': dataset[dataset.lockdown== lockdown][column1].mean()},
        value = dataset[dataset.lockdown== lockdown][column1].mean(),
#         title = {'text': column1},
        domain = {'x': [x1, x2], 'y': [y1, y2]},
        gauge = {
        'shape': "bullet",
        'axis': {'range': [None, dataset[column1].quantile(1.0)], 'tickwidth': 0.5, 'tickcolor': "cyan"},
        'threshold': {
            'line': {'color': "black", 'width': 2},
            'thickness': 0.75,
            'value': dataset[column1].mean()},
        'steps': [
            {'range': [0, 
                       marseille[column1].quantile(0.25)], 'color': 'ghostwhite'},
            {'range': [dataset[column1].quantile(0.25), 
                       dataset[column1].quantile(0.50)], 'color': 'yellow'},
            {'range': [dataset[column1].quantile(0.50),
                       dataset[column1].quantile(0.75)], 'color': 'gold'},
            {'range': [dataset[column1].quantile(0.75),
                      dataset[column1].quantile(1.0)], 'color': 'olive'}],
            'bar': {'color': "black"}
        }
            ))
    return fig_b



In [ ]:
def bulleten_air_pollutant_lockdowns(dataset,column2,lockdown,x1,x2,y1,y2):
    fig_b.add_trace(go.Indicator(
        mode = "number+gauge+delta",
        delta = {'reference': 3},
        value = dataset[dataset.lockdown== lockdown][column2].mean(),
        title = {'text': '- '+lockdown+' -             '},
        domain = {'x': [x1, x2], 'y': [y1, y2]},
        gauge = {
        'shape': "bullet",
        'axis': {'range': [None, dataset[column2].quantile(1.0)], 'tickwidth': 0.5, 'tickcolor': "cyan"},
        'threshold': {
            'line': {'color': "black", 'width': 2},
            'thickness': 0.75,
            'value': 3},
        'steps': [
            {'range': [0, 
                       1], 'color': 'ghostwhite'},
            {'range': [1, 
                       2], 'color': 'lavender'},
            {'range': [2,
                       3], 'color': 'cornflowerblue'},
            {'range': [3,
                      4], 'color': 'royalblue'},
            {'range': [4,
                       5], 'color': 'blue'},
            {'range': [5,
                       6], 'color': 'navy'}],
            'bar': {'color': "black"}
        }
            ))
    return fig_b



In [ ]:
def bulleten_compare(dataset,column1,column2):
    
    #################### before lockdown1 ############################
    fig_b = bulleten_mobility_lockdowns(dataset,column1,'before_lockdown1',0,0.3,0.9,1)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'before_lockdown1',0.7,1,0.9,1)

    #################### lockdown 1 ############################

    fig_b = bulleten_mobility_lockdowns(dataset,column1,'lockdown_1',0,0.3,0.7,0.775)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'lockdown_1',0.7,1,0.7,0.775)

    #################### before lockdown2 ############################

    fig_b = bulleten_mobility_lockdowns(dataset,column1,'before_lockdown2',0,0.3,0.6,0.675)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'before_lockdown2',0.7,1,0.6,0.675)

    #################### lockdown 2 ############################
    fig_b = bulleten_mobility_lockdowns(dataset,column1,'lockdown_2',0,0.3,0.4,0.475)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'lockdown_2',0.7,1,0.4,0.475)

    ####################before lockdown3 ############################

    fig_b = bulleten_mobility_lockdowns(dataset,column1,'before_lockdown3',0,0.3,0.3,0.375)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'before_lockdown3',0.7,1,0.3,0.375)

    #################### lockdown 3 ############################

    fig_b = bulleten_mobility_lockdowns(dataset,column1,'lockdown_3',0,0.3,0.1,0.175)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'lockdown_3',0.7,1,0.1,0.175)

    #################### after lockdown 3 ############################

    fig_b = bulleten_mobility_lockdowns(dataset,column1,'after_lockdown3',0,0.3,0,0.075)
    fig_b = bulleten_air_pollutant_lockdowns(dataset,column2,'after_lockdown3',0.7,1,0,0.075)

    
    fig_b.show()

## cities

** **
**Marseille**

In [ ]:
marseille = master_numeric[master_numeric.region == 'Marseille']

In [ ]:
# marseille.set_index(['date'],inplace=True)
marseille.head()

In [ ]:
color_lockdown = []
for i in marseille.lockdown:
    if i == 'No lockdown':
        color_lockdown.append('tomato')
    elif i == 'lockdown_1':
        color_lockdown.append('blue')
    elif i == 'lockdown_2':
        color_lockdown.append('lime')
    elif i == 'lockdown_3':
        color_lockdown.append('cyan')


air pollutant = excellent,fine,moderate,poor,very poor

o3 = 33,65,120,180,240

pm25 = 5,10,20,25,60

pm10 = 10,20,35,50,100

no2 = 10,20,35,50,100

so2 = 25,50,120,350,500

In [ ]:
marseille,color_no2,color_o3,color_pm10,color_pm25,color_so2 = air_pollutant_catagory_color(marseille)

In [ ]:
marseille.columns

In [ ]:
stacked_chart(marseille,color_no2,color_o3,color_pm10,color_pm25,color_so2)

In [ ]:
fig_g = go.Figure()
gauge_compare(marseille,'driving',"so2")

In [ ]:
fig_b = go.Figure()
bulleten_compare(marseille,'driving',"so2")

** **
**Nice**

In [ ]:
nice = master_numeric[master_numeric.region == 'Nice']

In [ ]:
# nice.set_index(['date'],inplace=True)
nice.head()

In [ ]:
nice,color_no2,color_o3,color_pm10,color_pm25,color_so2 = air_pollutant_catagory_color(nice)

In [ ]:
nice.columns

In [ ]:
stacked_chart(nice,color_no2,color_o3,color_pm10,color_pm25,color_so2)

In [ ]:
fig_g = go.Figure()
gauge_compare(nice,'walking',"pm10")

In [ ]:
fig_b = go.Figure()
bulleten_compare(nice,'driving',"no2")

** **
**Lyon**

In [ ]:
lyon = master_numeric[master_numeric.region == 'Lyon']

In [ ]:
lyon.set_index(['date'],inplace=True)
lyon.head()